# Serre-Faltings example

This example is the one which takes up most of the paper "On rational Bianchi newforms and abelian surfaces with
  quaternionic multiplication" by Cremona, Demb\'el\'e, Pacetti, Schembri and Voight.

### Define the base field:

In [1]:
K.<a> = NumberField(x^2-x+2)
K.discriminant()

-7

In [2]:
%runfile C2C3S3.py
%runfile T0T1T2.py

Define the set of bad primes of $K$, including the primes dividing 2:

In [3]:
S = K.ideal(2*5*7).prime_factors()
[(P,P.norm()) for P in S]

[(Fractional ideal (a), 2),
 (Fractional ideal (-a + 1), 2),
 (Fractional ideal (5), 25),
 (Fractional ideal (-2*a + 1), 7)]

## Step 1
### Find a set of primes $T_0$ such that the mod 2 representation is reducible if and only if the $a_P$ for $P\in T_0$ are all even, together with a set of cubics defining all $C_3$ and $S_3$ extensions of $K$ unramified outside $S$.
NB This takes time...used to be about 11m, now is over an hour (I do not yet know why)

In [4]:
%time flist = C3S3_extensions(K, K.ideal(2*5*7).prime_factors())

CPU times: user 1h 4min 56s, sys: 636 ms, total: 1h 4min 57s
Wall time: 1h 4min 56s


In [5]:
%time cubics, T0, vlist = get_T0(K,S,flist)

CPU times: user 549 ms, sys: 4 ms, total: 553 ms
Wall time: 551 ms


The cubics are

In [6]:
cubics

[x^3 + (-a - 1)*x^2 + (a - 2)*x + 1,
 x^3 - 2*a*x^2 + (-2*a - 1)*x - a - 2,
 x^3 + (2*a - 2)*x^2 + (2*a - 3)*x + a - 3,
 x^3 - a*x^2 + (a - 1)*x + 1,
 x^3 - a*x^2 + (a - 2)*x + a,
 x^3 + (-a - 1)*x^2 + (4*a - 3)*x - a + 3,
 x^3 - a*x^2 + (-3*a + 1)*x + 8*a - 24,
 x^3 + (-3*a + 1)*x^2 + (a - 4)*x - 2,
 x^3 + (-2*a + 1)*x^2 + (5*a - 6)*x - 5*a + 6]

The first 4 are $C_3$ and the last 5 are $S_3$

In [7]:
["C3" if c.discriminant().is_square() else "S3" for c in cubics]

['C3', 'C3', 'C3', 'C3', 'S3', 'S3', 'S3', 'S3', 'S3']

The test primes are

In [8]:
[(P,P.norm()) for P in T0]

[(Fractional ideal (-2*a + 3), 11),
 (Fractional ideal (2*a + 1), 11),
 (Fractional ideal (-2*a + 5), 23),
 (Fractional ideal (-4*a + 1), 29),
 (Fractional ideal (-6*a + 1), 67)]

So we need to know the parity of $a_P$ for these primes $P$ and match with one (unique) row of vlist, and that will tell us which cubic is ours.

We extract the Hecke eigenvalues manually from http://www.lmfdb.org/ModularForm/GL2/ImaginaryQuadratic/2.0.7.1/30625.1/e/

In [9]:
aPdata = {}
aPdata[K.ideal(3-2*a)] = 4
aPdata[K.ideal(1+2*a)] = 4
aPdata[K.ideal(5-2*a)] = -7
aPdata[K.ideal(1-4*a)] = 1
aPdata[K.ideal(1-6*a)] = 9
assert all([P in aPdata for P in T0])

Compute the test vector, check that it matches exactly one of the vectors in vlist, and return the associated cubic

In [10]:
test_vector = [aPdata[P]%2 for P in T0]
assert test_vector in vlist
assert len([v for v in vlist if v==test_vector])==1
cubic = next(cubic for cubic,vec in zip(cubics,vlist) if test_vector==vec)
Gtype = "C3" if cubic.discriminant().is_square else "S3"
print("Residual representation is irreducible, image is {}, kernel polynomial {}".format(Gtype,cubic))

Residual representation is irreducible, image is C3, kernel polynomial x^3 + (-a - 1)*x^2 + (4*a - 3)*x - a + 3


Make the polynomial absolute, optimise and factor over $K$ to get a nicer polynomial:

In [11]:
sextic = K.extension(cubic,'b').absolute_field('b').optimized_representation()[0].defining_polynomial()
sextic

x^6 - 3*x^5 + 2*x^4 - 6*x^3 + 25*x^2 - 19*x + 8

In [12]:
cubic = sextic.change_ring(K).factor()[0][0]
cubic

x^3 + (-a - 1)*x^2 + (4*a - 3)*x - a + 3

## Step 2
### Apply Faltings-Serre for $S_3$ residual image:

In [13]:
%runfile SerreFaltings.py

In [14]:
PP2 = S3primes(K,S,cubic)
PP2

[Fractional ideal (-2*a + 3),
 Fractional ideal (-4*a + 1),
 Fractional ideal (4*a - 7),
 Fractional ideal (3),
 Fractional ideal (2*a + 3),
 Fractional ideal (2*a + 1),
 Fractional ideal (-4*a - 3),
 Fractional ideal (-2*a + 5),
 Fractional ideal (-4*a + 5)]

We do not yet have a_P for some of these:

In [15]:
[(P,P.norm()) for P in PP2 if not P in aPdata]

[(Fractional ideal (4*a - 7), 53),
 (Fractional ideal (3), 9),
 (Fractional ideal (2*a + 3), 23),
 (Fractional ideal (-4*a - 3), 53),
 (Fractional ideal (-4*a + 5), 37)]

So we go back to the Bianchi form for them:

In [16]:
aPdata[K.ideal(4*a-7)] = 2
aPdata[K.ideal(3)] = 3
aPdata[K.ideal(2*a+3)] = 7
aPdata[K.ideal(4*a+3)] = -2
aPdata[K.ideal(4*a-5)] = 4
assert all([P in aPdata for P in PP2])